## Water Demand Tutorial

In [ ]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import folium
import scipy
import seaborn as sns
from sentinelsat import SentinelAPI, make_path_filter
from IPython import display
import shapely
from shapely.geometry import MultiPolygon

# Set Seaborn style and color palette
sns.set_style("darkgrid")
sns.set_palette("husl")

# generate matplotlib handles to create a legend of the features we put in our map.
def generate_handles(labels, colors, edge='k', alpha=1):
    lc = len(colors)  # get the length of the color list
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc], edgecolor=edge, alpha=alpha))
    return handles


# create a scale bar of length 20 km in the upper right corner of the map
# adapted this question: https://stackoverflow.com/q/32333870
# answered by SO user Siyh: https://stackoverflow.com/a/35705477
def scale_bar(ax, location=(0.92, 0.95)):
    x0, x1, y0, y1 = ax.get_extent()
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    ax.plot([sbx, sbx - 20000], [sby, sby], color='k', linewidth=9, transform=ax.projection)
    ax.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=6, transform=ax.projection)
    ax.plot([sbx-10000, sbx - 20000], [sby, sby], color='w', linewidth=6, transform=ax.projection)

    ax.text(sbx, sby-4500, '20 km', transform=ax.projection, fontsize=8)
    ax.text(sbx-12500, sby-4500, '10 km', transform=ax.projection, fontsize=8)
    ax.text(sbx-24500, sby-4500, '0 km', transform=ax.projection, fontsize=8)

In [ ]:
# load the outline of UK for a backdrop
outline = gpd.read_file(os.path.abspath('data_files/Outline.shp'))

In [ ]:
# Load water company data as wrz, remove unnecessary columns
wrz = gpd.read_file(os.path.abspath('data_files/WaterSupplyAreas_incNAVs v1_4.shp'))
# List of columns to be removed
columns_to_remove = ['Disclaimer', 'Disclaim2', 'Disclaim3', 'Provenance', 'Licence', 'WARNINGS', 'Revisions']

# Drop the columns from the GeoDataFrame
wrz = wrz.drop(columns=columns_to_remove)

In [ ]:
wrz.head(10)

In [ ]:
myFig = plt.figure(figsize=(10, 10))  # create a figure of size 10x10 (representing the page size in inches)

myCRS = ccrs.TransverseMercator(27700)  # create a Universal Transverse Mercator reference system to transform our data.

ax = plt.axes(projection=myCRS)  # finally, create an axes object in the figure, using a UTM projection,
# where we can actually plot our data.

In [ ]:
# first, we just add the outline of Northern Ireland using cartopy's ShapelyFeature
outline_feature = ShapelyFeature(outline['geometry'], myCRS, edgecolor='k', facecolor='w')
xmin, ymin, xmax, ymax = outline.total_bounds
ax.add_feature(outline_feature) # add the features we've created to the map.

In [ ]:
# using the boundary of the shapefile features, zoom the map to our area of interest
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=myCRS) # because total_bounds 
# gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.

myFig ## re-draw the figure

In [ ]:
# get the number of unique water companies we have in the dataset
num_wrz = len(wrz.COMPANY.unique())
print('Number of unique features: {}'.format(num_wrz))

In [ ]:
# pick colours for the individual water companies- generate random RGB colors
num_colours = num_wrz  # Number of colours to generate
company_colours = sns.color_palette("husl", n_colors=num_colours)  # Use Seaborn color palette

In [ ]:
# get a list of unique names for the company boundaries
company_names = list(wrz.COMPANY.unique())
company_names.sort() # sort the companies alphabetically by name

In [ ]:
# next, add the company outlines to the map using the colours that we've picked.
for ii, name in enumerate(company_names):
    feat = ShapelyFeature(wrz.loc[wrz['COMPANY'] == name, 'geometry'], # first argument is the geometry
                          myCRS, # second argument is the CRS
                          edgecolor='k', # outline the feature in black
                          facecolor=company_colours[ii], # set the face color to the corresponding color from the list
                          linewidth=1, # set the outline width to be 1 pt
                          alpha=0.25) # set the alpha (transparency) to be 0.25 (out of 1)
    ax.add_feature(feat) # once we have created the feature, we have to add it to the map using ax.add_feature()

In [ ]:
#myFig # to show the updated figure

In [ ]:
# Add vertical gridlines
ax.vlines(x=[100000, 200000, 300000, 400000, 500000, 600000, 700000],
          ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1],
          color='gray', linestyle=':', linewidth=0.5)

# Add horizontal gridlines
ax.hlines(y=[100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000],
          xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1],
          color='gray', linestyle=':', linewidth=0.5)

# Add labels to x-axis
x_ticks = [100000, 200000, 300000, 400000, 500000, 600000, 700000]
for x in x_ticks:
    ax.text(x, ax.get_ylim()[0] - 50000, str(x), ha='center', va='top', fontsize=8)

# Add labels to y-axis
y_ticks = [100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000]
for y in y_ticks:
    ax.text(ax.get_xlim()[0] - 50000, y, str(y), ha='right', va='center', fontsize=8)

ax.set_xlabel('X Axis Label')
ax.set_ylabel('Y Axis Label')

myFig # to show the updated figure

In [ ]:
print(wrz.crs == outline.crs) # test if the crs is the same 

In [ ]:
wrz.loc[1]

In [ ]:
# Append PCC for 2019 to 2020 to the wrz geodataframe

# Load the CSV file
pr24_hist_pcc = pd.read_csv('data_files/pr24_hist_pcc.csv')  

# Perform the merge
merged = wrz.merge(pr24_hist_pcc[['Company', '2019-20']], how='left', left_on='Acronym', right_on='Company')

# Drop the unnecessary columns & rename the merged column
merged.drop(['Company'], axis=1, inplace=True)
merged.rename(columns={'2019-20': '2019-20_from_CSV'}, inplace=True)

# Update the wrz GeoDataFrame with the merged column
wrz['2019-20'] = merged['2019-20_from_CSV']

In [ ]:
#Add chloropleth map
# Create the chloropleth map
wrz.plot(column='2019-20', cmap='viridis', linewidth=0.8, edgecolor='black', legend=True, figsize=(10, 10))

# Set plot title and axis labels
plt.title('Chloropleth Map of 2019-20')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

In [ ]:
wrz.loc[1]

In [ ]:
# Append PCC for 2011 to 2012 to the wrz geodataframe
# Load the GeoDataFrame
# wrz = gpd.read_file(os.path.abspath('data_files/WaterSupplyAreas_incNAVs v1_4.shp'))  

# Load the CSV file
pr24_hist_pcc = pd.read_csv('data_files/pr24_hist_pcc.csv') 

# Perform the merge
merged = wrz.merge(pr24_hist_pcc[['Company', '2011-12']], how='left', left_on='Acronym', right_on='Company')

# Drop the unnecessary columns
merged.drop(['Company'], axis=1, inplace=True)

# Rename the merged column
merged.rename(columns={'2011-12': '2011-12_from_CSV'}, inplace=True)

# Update the wrz GeoDataFrame with the merged column
wrz['2011-12'] = merged['2011-12_from_CSV']

# Update the wrz GeoDataFrame with the merged column
wrz['2011-12'] = merged['2011-12_from_CSV']

# Check the updated GeoDataFrame
wrz.loc[1]

In [ ]:
# Create the chloropleth map
wrz.plot(column='2019-20', cmap='viridis', linewidth=0.8, edgecolor='black', legend=True, legend_kwds={'title': 'PCC in litres per day'}, figsize=(10, 10))

# Set plot axis labels
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Add a border around the map
for spine in plt.gca().spines.values():
    spine.set_visible(True)
    spine.set_color('black')
    spine.set_linewidth(1)

# Set plot title at the bottom of the map
plt.suptitle('Figure x: Average per capital consumption per water company area for 2019-20', y=0.1, fontsize=14)

# Adjust the spacing between the title and the axis labels
plt.subplots_adjust(bottom=0.175)

# Show the plot
plt.show()

In [ ]:
legend_kwds={'caption': 'Per Capita Water Consumption'} # set the caption to a longer explanation

In [ ]:
# Create a Folium map using the existing GeoDataFrame
m = folium.Map()

m = wrz.explore('2019-20', # show the PCC for this period column
                   cmap='viridis', # use the 'plasma' colormap from matplotlib
                   legend_kwds={'caption': 'PCC for 2019-20'} # set the caption to a longer explanation
                  )

airport_args = {
    'm': m, # add the markers to the same map we just created
    'marker_type': 'marker', # use a marker for the points, instead of a circle
    'popup': True, # show the information as a popup when we click on the marker
    'legend': False, # don't show a separate legend for the point layer
    'marker_kwds': {'icon': folium.Icon(color='red', icon='dot', prefix='fa')} # make the markers red with a dot icon from FA
}
m

In [ ]:
# install sentinelsat via conda; set up netrc file for api

In [ ]:
# select water company to review in further detail:
selected_company_gdf = wrz[wrz['AreaServed'] == 'Bournemouth']

# get the outline of the selected water company:
Bournemouth = selected_company_gdf['geometry']

# create a new GeoDataFrame with the outline of the selected water company:
bournemouth_gdf = gpd.GeoDataFrame(geometry=Bournemouth)

# print the GeoDataFrame
print(bournemouth_gdf)

In [ ]:
# bournemouth_gdf to epsg 4326
bournemouth_gdf = bournemouth_gdf.to_crs(epsg=4326)

# convert the MULTIPOLYGON to a valid POLYGON
polygon = bournemouth_gdf['geometry'].unary_union
polygon 

In [ ]:
# get the minimum rotated angle
search_area = polygon.minimum_rotated_rectangle

search_area

In [ ]:
api = SentinelAPI(None, None, api_url='https://scihub.copernicus.eu/dhus')

In [ ]:
products = api.query(search_area.wkt, # use the WKT representation of our search area
                     date=('20230201', '20230228'), # all images from February 2023
                     platformname='Sentinel-2', # the platform name is Sentinel-2
                     producttype='S2MSI2A', # surface reflectance product (L2A)
                     cloudcoverpercentage=(0, 10)) # limit to 10% cloud cover

In [ ]:
nresults = len(products) # get the number of results found
print('Found {} results'.format(nresults)) # show the number of results found 

In [ ]:
results = list(products)[1] # gets the second item from the dict
products[results] # show the metadata for the second item

In [ ]:
qlook = api.download_quicklook(results) # download the quicklook image for this first result 
display.Image(qlook['path']) # display the image 

In [ ]:
product_geo = SentinelAPI.to_geodataframe(products) # convert the search results to a geodataframe
product_geo.head() # show the first 5 rows of the geodataframe

In [ ]:
for ind, row in product_geo.iterrows():
    intersection = search_area.intersection(row['geometry']) # find the intersection of the two polygons
    product_geo.loc[ind, 'overlap'] = intersection.area / search_area.area # get the fractional overlap
    
print(product_geo.overlap) # show the fractional overlap for each index

In [ ]:
max_index = product_geo.overlap.argmax() # get the integer location of the largest overlap value
print(max_index) 

In [ ]:
best_overlap = product_geo.inbest_overlap = product_geo.index[max_index] # get the actual index (image name) with the largest overlap
print(product_geo.loc[best_overlap]) # show the metadata for the image with the largest overlap

In [ ]:
qlook = api.download_quicklook(best_overlap) # download the quicklook image for the first result
display.Image(qlook['path']) # display the image

In [ ]:
api.download_all(products,
    n_concurrent_dl=5,  # allow 5 concurrent downloads
    nodefilter=make_path_filter("_B*.jp2")) #only download the image bands